# Events Suggesting

## Setup

start_date = '2024-03-24'
end_date = '2024-03-24'

url = f'http://events.umich.edu/list/json?filter=all&range={start_date}to{end_date}&v=2'

with urllib.request.urlopen(url) as response:
    data = response.read()

events_json = json.loads(data)

event_descriptions = [event["description"] for event in events_json]

context = {
    "location_name": itemgetter("location_name"),
    "room": itemgetter("room"),
    "event_type": itemgetter("event_type"),
    "event_title": itemgetter("event_title")
}

vectorstore = FAISS.from_texts(event_descriptions, embedding=OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY")))


retriever = vectorstore.as_retriever(k=5)

template = """Provide information based on the event descriptions:
{context}

Question: {question}
"""

model = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model="gpt-3.5-turbo")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | PromptTemplate.from_template(template, context=context)
    | model
    | StrOutputParser()
)


chain.invoke("What  music events happening today")

In [1]:
from langchain_community.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os
import json
import urllib.parse
import urllib.request
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
load_dotenv()


True

In [2]:
# os.environ['OPENAI_API_KEY'] = ''
# print(os.getenv("OPENAI_API_KEY"))
model = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model="gpt-3.5-turbo")

/Users/ellazhang/Desktop/MDST/mdst-events-suggester-llm/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## Functions

# functions we might need:
# -function for getting events, filtered on date and type of event
### -RAG/question answer type thing from the top of langchain-rag.ipynb
#### - determining which events the user is most interest in via vector embedding search
# - function for getting date 
# - function for getting/asking for type of event? 


In [4]:
from langchain.tools import tool
import dateparser
from daterangeparser import parse

@tool
def get_date(input_expr: str) -> str:
    """Function that, given date information in weekday or date formats about when a user is interested in going to an event, returns a date in year-month-day format. Do not include 'this' or 'next' in input string. If the input is too vague, the function returns -1. This function should be called multiple times in order to get start and end dates of the time range the user is interested in."""
    if input_expr.split(" ")[0] == "this":
        input_expr = input_expr[5:] 
    # if input_expr.split(" ")[0] == "next":
    #     input_expr = input_expr[5:] 
   
    date = dateparser.parse(input_expr, settings={'PREFER_DATES_FROM': 'future'})
    if (date is None):
        return -1
    return str(date.date())



In [31]:
tools = [get_date]
system_message = f"""
    You are a chatbot helper that helps users find events happening around Ann Arbor that they are interested in going to.

    You must ask them for the dates they're interested in going to and the kinds of events they are interested in. 
    You must then suggest events happening in the given date range that they would be interested in. 

    You have several tools at your disposal, which you will have to take advantage of to figure out when the user is interested in going to an event and what kinds of events they like.
    - You can leverage some functions for:
        - Figuring out the specific date range the user is interested in given inputs in day-of-week or date formats. If the function returns -1, ask the user to provide more specific date information. This function should be called multiple times in order to get both start and end dates of the time range the user is interested in.
        - Getting information about events the user might be interested in that are happening during the dates they are interested in.
    - You can leverage some external resources for:
        - You can ask the user for more information, such as what exact dates they're interested in or what kinds of events they are interested in.
    """

from langchain.memory import ChatMessageHistory

history = []


In [257]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="chat-conversational-react-description",
    tools=tools,
    llm=model,
    agent_kwargs={
        "system_message": system_message,
    },
    verbose=True,
    max_iterations=3
)

zero_shot_agent.invoke({"input": "what are some concerts happening the", "chat_history": history})





> Entering new AgentExecutor chain...
```json
{
    "action": "get_date",
    "action_input": "the day after tomorrow"
}
```the day after tomorrow

Observation: 2024-03-26
Thought:```json
{
    "action": "Final Answer",
    "action_input": "2024-03-26"
}
```

> Finished chain.


{'input': 'what are some concerts happening the day after tomorrow.',
 'chat_history': [],
 'output': '2024-03-26'}

In [29]:
start_date = '2024-03-24'
end_date = '2024-03-24'

url = f'http://events.umich.edu/list/json?filter=all&range={start_date}to{end_date}&v=2'

with urllib.request.urlopen(url) as response:
    data = response.read()

events_json = json.loads(data)

event_descriptions = []
for event in events_json:
    if event["combined_title"] == None:
        event["combined_title"] = "An event"
    if event["location_name"] == None:
        event["location_name"] = " an unknown location"
    
    if event["room"] != "":
        new_event = (event["combined_title"] + " is happening in " +  event["location_name"] + " " + event["room"])
    else: 
        new_event = (event["combined_title"] + " is happening in " +  event["location_name"])

    if event["date_start"] == event["date_end"]:
        new_event += ". The date of the event is " + event["date_start"] + "."
    else:
        new_event += ". The dates of the event are from " + event["date_start"] + " to " + event["date_end"] + "."
    
    if event["time_start"] == "00:00:00" and event["time_end"] == "23:59:59":
        new_event += "The event is happening all day."
    else:
        new_event += ". The time the event is happening is from " + event["time_start"] + " to " + event["time_end"] + "."
    

    new_event += ". A description of the event is: " + event["description"]

    if event["cost"] != "":
        new_event += ". Cost information is: " + event["cost"]

    event_descriptions.append(new_event)
# context = {
#     "location_name": itemgetter("location_name"),
#     "room": itemgetter("room"),
#     "event_type": itemgetter("event_type"),
#     "event_title": itemgetter("event_title")
# }}

vectorstore = FAISS.from_texts(event_descriptions, embedding=OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY")))


retriever = vectorstore.as_retriever(k=5)

template = """Provide information about events the user might be interested in, including location, date, and time information about those events:
{context}

Question: {question}
"""

model = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model="gpt-3.5-turbo")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | PromptTemplate.from_template(template)
    | model
    | StrOutputParser()
)


print(chain.invoke("What music events are happening today ?"))

Today, there are several music events happening at the University of Michigan. Here are some details:

1. Jazz Piano Studio Recital:
   - Location: Earl V. Moore Building Britton Recital Hall
   - Date: March 24, 2024
   - Time: 7:00 PM to 8:30 PM
   - Description: Duo and solo piano performances by students from the studio of Professor Andy Milne, featuring a jazz piano journey from Chopin to Blondie.
   - Cost: Free

2. Davide Jeon, jazz piano:
   - Location: Earl V. Moore Building McIntosh Theatre
   - Date: March 24, 2024
   - Time: 12:00 PM to 1:00 PM
   - Description: Undergraduate student Davide Jeon's piano recital.
   - Cost: Free

3. Paul Nastelin, French horn:
   - Location: Earl V. Moore Building Britton Recital Hall
   - Date: March 24, 2024
   - Time: 11:00 AM to 12:00 PM
   - Description: Undergraduate student Paul Nastelin's French horn recital.
   - Cost: Free

4. Jolie Rebelo, violin:
   - Location: Walgreen Drama Center Stamps Auditorium
   - Date: March 24, 2024
   